# *FEniCS tutorial:* Heat equation with Dirichlet boundary conditions

In this demo, we solve the two-dimensional diffusion equation with Dirichlet
boundary conditions $u_D$ and source term $f$. Both are chosen so as to yield
an exact analytic result against which we can compare the numerical results.

$$
\begin{align}
  u'   &= \nabla^2 u + f     \quad\text{in the unit square} \\
  u    &= u_D  \hphantom{u+f}\quad\text{on the boundary} \\
  u    &= u_0  \hphantom{u+f}\quad\;\text{at $t = 0$}
\end{align}
$$
with
$$
\begin{align}
  u_D  &= 1 + x^2 + \alpha y^2 + \beta t \\
  u_0 &= u_D(t=0) \\[0.5ex]
  f    &= \beta - 2 (1 + \alpha)
\end{align}
$$

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time as tm

# import os
# import re
# from color import *
# from IPython.display import Image
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# from mplotg import *
# import matplotlib.pyplot as plt

def multiplot_from_generator(g, num_columns, figsize_for_one_row=None):
    # default to 15-inch rows, with square subplots
    if figsize_for_one_row is None:
        figsize_for_one_row = (15, 15/num_columns)
    try:
        while True:
            # call plt.figure once per row
            plt.figure(figsize=figsize_for_one_row)
            for col in range(num_columns):
                ax = plt.subplot(1, num_columns, col+1)
                next(g)
    except StopIteration:
        pass

In [ ]:
from fenics import *

In [ ]:
## set up the problem and define main simulation function evolve()

# set parameters
T = 2.0            # total simulation time
num_steps = 10     # number of time steps
dt = T / num_steps # size of time step
alpha = 3          # parameter alpha
beta = 1.2         # parameter beta

# create mesh and define function space
nx = ny = 8
mesh = UnitSquareMesh(nx, ny)
V = FunctionSpace(mesh, 'P', 1)

# define boundary condition
u_D = Expression('1 + x[0]*x[0] + alpha*x[1]*x[1] + beta*t',
                 degree=2, alpha=alpha, beta=beta, t=0)

def boundary(x, on_boundary):
    return on_boundary

bc = DirichletBC(V, u_D, boundary)

# define initial value
u_n = interpolate(u_D, V)
#u_n = project(u_D, V)

# define variational problem
u = TrialFunction(V)
v = TestFunction(V)
f = Constant(beta - 2 - 2*alpha)
F = u*v*dx + dt*dot(grad(u), grad(v))*dx - (u_n + dt*f)*v*dx
a, L = lhs(F), rhs(F)

# define time-evolution function
def evolve():
    # compute and report initial error at vertices
    u_e = interpolate(u_D, V)
    error = np.abs(u_e.vector() - u_n.vector()).max()
    print('n = %2d: t = %.2f: error = %.3g' % (0, 0., error))

    # plot initial state
    yield plot(u_n)

    # time-stepping
    nip =  1 # number of intervals between plots
    u = Function(V)
    t = 0
    for n in range(1, num_steps):
    
        # update current time
        t += dt
        u_D.t = t

        # compute solution
        solve(a == L, u, bc)

        # plot solution at nip-step intervals
        if n % nip == 0:
            yield plot(u)

        # compute and report error at vertices
        u_e = interpolate(u_D, V)
        error = np.abs(u_e.vector() - u.vector()).max()
        print('n = %2d: t = %.2f: error = %.3g' % (n, t, error))

        # update previous solution
        u_n.assign(u)

In [ ]:
plots = evolve()

In [ ]:
n_rows = 3
n_cols = 5
fig_wd = 15
# default sizing here yields unit aspect ratio
plt.figure(figsize = (fig_wd, fig_wd * n_rows // n_cols))

#idx = 0
#for row in range(n_rows):
#    for col in range(n_cols):
#        idx += 1
#        ax = plt.subplot(n_rows, n_cols, idx)
#        next(g)

idx = 0
for p in evolve():
    idx += 1
    print('idx = %2d' % (idx))
    ax = plt.subplot(n_rows, n_cols, idx)

In [ ]:
## set up the problem and define main simulation function evolve()

# set parameters
T = 2.0            # total simulation time
num_steps = 10     # number of time steps
dt = T / num_steps # size of time step
alpha = 3          # parameter alpha
beta = 1.2         # parameter beta

# create mesh and define function space
nx = ny = 8
mesh = UnitSquareMesh(nx, ny)
V = FunctionSpace(mesh, 'P', 1)

# define boundary condition
u_D = Expression('1 + x[0]*x[0] + alpha*x[1]*x[1] + beta*t',
                 degree=2, alpha=alpha, beta=beta, t=0)

def boundary(x, on_boundary):
    return on_boundary

bc = DirichletBC(V, u_D, boundary)

# define initial value
u_n = interpolate(u_D, V)
#u_n = project(u_D, V)

# define variational problem
u = TrialFunction(V)
v = TestFunction(V)
f = Constant(beta - 2 - 2*alpha)
F = u*v*dx + dt*dot(grad(u), grad(v))*dx - (u_n + dt*f)*v*dx
a, L = lhs(F), rhs(F)

# compute and report initial error at vertices
u_e = interpolate(u_D, V)
error = np.abs(u_e.vector() - u_n.vector()).max()
print('n = %2d: t = %.2f: error = %.3g' % (0, 0., error))

# time-stepping
nip =  1 # number of intervals between plots
u = Function(V)
t = 0
for n in range(1, num_steps):

    # update current time
    t += dt
    u_D.t = t

    # compute solution
    solve(a == L, u, bc)

    # compute and report error at vertices
    u_e = interpolate(u_D, V)
    error = np.abs(u_e.vector() - u.vector()).max()
    print('n = %2d: t = %.2f: error = %.3g' % (n, t, error))

    # update previous solution
    u_n.assign(u)

In [ ]:
multiplot_from_generator(evolve(), 5, (15, 3))

In [ ]:
sin(3.14159)
sqrt(3.14159)
np.arange(3)

In [ ]:
# define time-evolution function
def sinc(x):
    if abs(x) < 2.e-8:
        return 1.
    else:
        return (sin(x) / x)

def sincs(a, b, ns):
    xvals = 
    for n in range(ns+1):
        xs = 

        # update current time
        t += dt
        u_D.t = t

        # Compute solution
        solve(a == L, u, bc)

        # Plot solution
        yield plot(u)

        # Compute error at vertices
        u_e = interpolate(u_D, V)
        error = np.abs(u_e.vector() - u.vector()).max()
        print('t = %.2f: error = %.3g' % (t, error))

        # Update previous solution
        u_n.assign(u)

In [ ]:
# time-stepping
u = Function(V)
t = 0
u_D.t = t
u_n = interpolate(u_D, V)
for n in range(num_steps):

    # update current time
    t += dt
    u_D.t = t

    # compute solution
    solve(a == L, u, bc)

    # plot solution
    plot(u)

    # compute error at vertices
    u_e = interpolate(u_D, V)
    error = np.abs(u_e.vector() - u.vector()).max()
    print('t = %.2f: error = %.3g' % (t, error))

    # update previous solution
    u_n.assign(u)

In [ ]:
a = -2.25 * pi
b =  2.25 * pi
ns = 256
xvals = np.arange(ns + 1) / ns
ovals = a * (1 - xvals) + b * xvals

def psinc(a, b, alf0, alf1, nalf):
    for j in range(0, nalf + 1):
        alf = alf0 * (1 - j / nalf) + alf1 * j / nalf
        yvals = np.sinc(alf * ovals)
        yield plt.plot(avals, yvals)

In [ ]:
n_rows = 2
n_cols = 2
fig_wd = 6
# default sizing here yields unit aspect ratio
plt.figure(figsize = (fig_wd, fig_wd * n_rows // n_cols))

#idx = 0
#for row in range(n_rows):
#    for col in range(n_cols):
#        idx += 1
#        ax = plt.subplot(n_rows, n_cols, idx)
#        next(g)

idx = 0
for p in evolve():
    idx += 1
    ax = plt.subplot(n_rows, n_cols, idx)